In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [8]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=14):  # Set num_classes to 14 for your custom dataset
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [9]:
# Hyperparameters
batch_size = 128
learning_rate = 0.01
num_epochs = 20

In [13]:
train_dir = '../data/Train'  # Adjust to your absolute path if needed
test_dir = '../data/Test'

In [18]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [19]:
train_dataset = ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [20]:
model = AlexNet(num_classes=14)  # Adjust to match the number of classes in your dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)


In [21]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

Epoch [1/20], Loss: 1.9284
Epoch [2/20], Loss: 0.7672
Epoch [3/20], Loss: 0.4957
Epoch [4/20], Loss: 0.4022
Epoch [5/20], Loss: 0.3316
Epoch [6/20], Loss: 0.2932
Epoch [7/20], Loss: 0.2444
Epoch [8/20], Loss: 0.2043
Epoch [9/20], Loss: 0.1967
Epoch [10/20], Loss: 0.1659
Epoch [11/20], Loss: 0.1577
Epoch [12/20], Loss: 0.1352
Epoch [13/20], Loss: 0.1237
Epoch [14/20], Loss: 0.1158
Epoch [15/20], Loss: 0.0980
Epoch [16/20], Loss: 0.0930
Epoch [17/20], Loss: 0.0743
Epoch [18/20], Loss: 0.0814
Epoch [19/20], Loss: 0.0592
Epoch [20/20], Loss: 0.0681


In [22]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 97.15%


In [26]:
# Save the trained model
model_save_path = './alexnet_model.pth'  # Specify the path where you want to save the model
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to ./alexnet_model.pth
